In [13]:
from keras import layers
import tensorflow as tf
import pandas as pd
import numpy as np

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)


df = pd.read_csv('../csv_dateien/starcraftFinalcsvs/allRunsButOnlyTerran.csv')

conditions = [
    (df['fraction'] == "Protoss"), #1
    (df['fraction'] == "Terraner"), #2
    (df['fraction'] == "Zerg"), #3
    (df['fraction'] == "T"), #2
]

values = [1, 2, 3, 2]

df["fraction"] = np.select(conditions, values)
df=df[df["total_army"]>0]
df=df[df["fraction"]==2]
df=df[df["supply"]<=200]

dfPrepSample = df.drop(columns=["player","winner","replay_filename", "Unnamed: 0", "total_army_value","fraction"])
len(dfPrepSample)


3009

In [14]:
df_sample = dfPrepSample.sample(frac=1, random_state=12)
dfTrainSample = df_sample.sample(frac=0.8)

dfTrainSample.head()

,total_workers,resource_mining,total_army,supply,time
1271,50,17817,59.0,126.0,643
2038,14,1889,15.0,31.0,288
3780,44,14148,22.0,78.0,473
2750,48,12742,52.0,118.0,575
4345,61,17874,41.0,117.0,687


In [15]:
dfTestSample = df_sample.drop(dfTrainSample.index)

dfTestSample.head()

,total_workers,resource_mining,total_army,supply,time
5386,15,1850,8.0,31.0,203
3947,49,26408,92.0,150.0,808
4023,44,10282,25.0,86.0,483
3646,67,20336,67.0,157.0,759
3246,53,15602,30.0,85.0,556


In [16]:
dfTrainFeatures = dfTrainSample.copy()
dfTrainFeatures

,total_workers,resource_mining,total_army,supply,time
1271,50,17817,59.0,126.0,643
2038,14,1889,15.0,31.0,288
3780,44,14148,22.0,78.0,473
2750,48,12742,52.0,118.0,575
4345,61,17874,41.0,117.0,687
...,...,...,...,...,...
1946,71,27184,73.0,165.0,770
5566,50,13963,26.0,86.0,527
4426,49,16416,59.0,118.0,658
3359,65,50732,107.0,180.0,1019


In [17]:

dfTrainLabels = dfTrainFeatures.pop("total_army")
dfTrainFeatures = np.array(dfTrainFeatures)
dfTrainFeatures

array([[   50., 17817.,   126.,   643.],
       [   14.,  1889.,    31.,   288.],
       [   44., 14148.,    78.,   473.],
       ...,
       [   49., 16416.,   118.,   658.],
       [   65., 50732.,   180.,  1019.],
       [   77., 50807.,   187.,  1073.]])

In [18]:
# normalize = layers.Normalization()
# normalize.adapt(dfTrainFeatures)

In [19]:
sc_model = tf.keras.Sequential([layers.Dense(5, activation="relu"),layers.ReLU(),layers.Dense(5, activation="relu"),layers.ReLU(), layers.Dense(1)]) # letzte lösung 30/5

sc_model.compile(loss = tf.keras.losses.MeanSquaredError(),
                      optimizer = tf.keras.optimizers.Adam())

In [20]:
sc_model.fit(dfTrainFeatures, dfTrainLabels, epochs=500)  #350

Epoch 1/500
76/76 [==============================] - 0s 628us/step - loss: 164721.3438
Epoch 2/500
76/76 [==============================] - 0s 600us/step - loss: 1077.4929
Epoch 3/500
76/76 [==============================] - 0s 580us/step - loss: 572.3341
Epoch 4/500
76/76 [==============================] - 0s 587us/step - loss: 563.9604
Epoch 5/500
76/76 [==============================] - 0s 580us/step - loss: 554.8777
Epoch 6/500
76/76 [==============================] - 0s 587us/step - loss: 545.2914
Epoch 7/500
76/76 [==============================] - 0s 587us/step - loss: 531.0328
Epoch 8/500
76/76 [==============================] - 0s 604us/step - loss: 517.7398
Epoch 9/500
76/76 [==============================] - 0s 600us/step - loss: 507.3267
Epoch 10/500
76/76 [==============================] - 0s 601us/step - loss: 489.1128
Epoch 11/500
76/76 [==============================] - 0s 613us/step - loss: 476.0233
Epoch 12/500
76/76 [==============================] - 0s 587us/step - 

In [21]:
sc_model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 5)                 25        
                                                                 
 re_lu_1 (ReLU)              (None, 5)                 0         
                                                                 
 dense_4 (Dense)             (None, 5)                 30        
                                                                 
 re_lu_2 (ReLU)              (None, 5)                 0         
                                                                 
 dense_5 (Dense)             (None, 1)                 6         
                                                                 
Total params: 61
Trainable params: 61
Non-trainable params: 0
_________________________________________________________________


In [22]:
dfTestFeatures = dfTestSample.copy()

dfTestLabels = dfTestFeatures.pop("total_army")
dfTestFeatures = np.array(dfTestFeatures)
dfTestLabels.head()

5386     8.0
3947    92.0
4023    25.0
3646    67.0
3246    30.0
Name: total_army, dtype: float64

In [23]:
from sklearn.metrics import r2_score


predictedValues = sc_model.predict(dfTestFeatures)
r2_score_sc = (r2_score(dfTestLabels, predictedValues))
stri = ""
if(r2_score_sc<0):
    stri = "negative" + str(r2_score_sc)
else:
    stri = str(r2_score_sc)

print(f"The R^2 value in the test set is {stri}")

19/19 [==============================] - 0s 556us/step
The R^2 value in the test set is 0.9094662682376589


In [24]:
sc_model.save('sc_ts_model.h5')
